In [14]:
import pandas as pd
import numpy as np
# 对实验3的数据集进行预处理

df1 = pd.read_csv('3_data.csv',encoding='utf-8')

df1 = df1.dropna()

df1 = df1.reset_index(drop=True)

df1 = df1.drop('id', axis=1)

df1['diagnosis'] = df1['diagnosis'].apply(lambda x: 0 if x == 'B' else 1)#.apply()方法将函数应用于指定位置的每个元素


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Split data
X = df1.drop('diagnosis', axis=1)
le = LabelEncoder()
y = le.fit_transform(df1['diagnosis'])

# Split data
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.67, random_state=42)

# Define classifiers
classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=3000),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
}

# Train and evaluate classifiers
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(f"Classification Report for {name} on Validation Set:")
    print(classification_report(y_val, y_pred))
    
    # After tuning parameters on validation set, evaluate on test set
    y_pred_test = clf.predict(X_test)
    print(f"Classification Report for {name} on Test Set:")
    print(classification_report(y_test, y_pred_test))

Classification Report for Logistic Regression on Validation Set:
              precision    recall  f1-score   support

           0       1.00      0.91      0.95        11
           1       0.88      1.00      0.93         7

    accuracy                           0.94        18
   macro avg       0.94      0.95      0.94        18
weighted avg       0.95      0.94      0.94        18

Classification Report for Logistic Regression on Test Set:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        24
           1       0.92      0.86      0.89        14

    accuracy                           0.92        38
   macro avg       0.92      0.91      0.91        38
weighted avg       0.92      0.92      0.92        38

Classification Report for Random Forest on Validation Set:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00         7


In [22]:
import pandas as pd
import numpy as np

# 对实验4的数据集进行预处理
df = pd.read_csv('4_data.csv')

df = df.dropna()

df = df.drop('id', axis=1)

replace_values1 = {'14-Oct': '10-14', '9-May': '5-9'}
df['tumor-size'] = df['tumor-size'].replace(replace_values1)

replace_values2 = {'5-Mar': '3-5', '8-Jun': '6-8', '11-Sep': '9-11', '14-Dec': '12-14'}
df['inv-nodes'] = df['inv-nodes'].replace(replace_values2)

excel = pd.read_excel('4_variables.xlsx', index_col=0)
temp1_variables = excel['Description'].tolist()
temp2_variables = [item.split(', ') for item in temp1_variables]
variables = []
for item in temp2_variables:
    for sublist in item:
        variables.append(sublist)

for column in df.columns[1:]:
    if column == 'irradiat':
        for i in df.index:
            first_index = variables.index(df.loc[i, column])
            second_index = variables[first_index+1:].index(df.loc[i, column]) + first_index + 1
            df.loc[i, column] = second_index
    else:
        for i in df.index:
            df.loc[i, column] = variables.index(str(df.loc[i, column]))
            
ind2val = {}

for attribute, row in excel.iterrows():
    if attribute != 'irradiat':
        for item in row['Description'].split(', '):
            index = variables.index(item)
            ind2val[index] = f"{attribute}={item}"
    else:
        for item in row['Description'].split(', '):
            first_index = variables.index(item)
            second_index = variables[first_index+1:].index(item) + first_index + 1
            index = second_index
            ind2val[index] = f"{attribute}={item}"

print(df)

                    Class age menopause tumor-size inv-nodes node-caps  \
0    no-recurrence-events   4        13         20        26        40   
1    no-recurrence-events   5        13         18        26        40   
2    no-recurrence-events   5        13         18        26        40   
3    no-recurrence-events   7        12         17        26        40   
4    no-recurrence-events   5        13         14        26        40   
..                    ...  ..       ...        ...       ...       ...   
281     recurrence-events   4        13         20        26        40   
282     recurrence-events   4        13         18        26        40   
283     recurrence-events   7        12         18        26        40   
284     recurrence-events   5        12         20        27        40   
285     recurrence-events   6        12         20        27        40   

     deg-malig breast breast-quad irradiat  
0           43     44          47       52  
1           42     45

In [27]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Prepare data
X = df.drop('Class', axis=1)
le = LabelEncoder()
y = le.fit_transform(df['Class'])

# Define classifiers
classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier()
}

# 使用k折交叉验证法划分训练集/测试集（可调整随机种子以获得不同的划分方案）
# Define k-fold cross validation
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# Perform k-fold cross validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Train and evaluate classifiers
    for name, clf in classifiers.items():
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(f"Classification Report for {name} on Test Set:")
        print(classification_report(y_test, y_pred))

Classification Report for Logistic Regression on Test Set:
              precision    recall  f1-score   support

           0       0.73      0.95      0.82        37
           1       0.75      0.32      0.44        19

    accuracy                           0.73        56
   macro avg       0.74      0.63      0.63        56
weighted avg       0.74      0.73      0.69        56

Classification Report for Random Forest on Test Set:
              precision    recall  f1-score   support

           0       0.71      0.81      0.76        37
           1       0.50      0.37      0.42        19

    accuracy                           0.66        56
   macro avg       0.61      0.59      0.59        56
weighted avg       0.64      0.66      0.65        56

Classification Report for Logistic Regression on Test Set:
              precision    recall  f1-score   support

           0       0.78      0.90      0.84        42
           1       0.43      0.21      0.29        14

    accurac